In [ ]:
!pip install torch==2.4.1 torchvision==0.19.1 --extra-index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.4/883.4 kB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -U git+https://github.com/luca-medeiros/lang-segment-anything.git

  Cloning https://github.com/luca-medeiros/lang-segment-anything.git to /tmp/pip-req-build-iad90g1d
  Running command git clone --filter=blob:none --quiet https://github.com/luca-medeiros/lang-segment-anything.git /tmp/pip-req-build-iad90g1d
  Resolved https://github.com/luca-medeiros/lang-segment-anything.git to commit 918043ed4666eea04da88aa179eb8d27ef4b1a1d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/facebookresearch/segment-anything-2 (to revision c2ec8e14a185632b0a5d8b161928ceb50197eddc) to /tmp/pip-install-grawnsoh/sam-2_4f763a02d97a4726b4befce08bbf5fe6
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything-2 /tmp/pip-install-grawnsoh/sam-2_4f763a02d97a4726b4befce08bbf5fe6
  Running command git rev-parse -q --verify 'sha^c2ec8e14a185632b0a5d8b161928ceb50197eddc'
  Running command git fetch -q https://github.co

In [ ]:
# save as run_langsam_all_masks.py
from PIL import Image
import numpy as np
import torch
from datetime import datetime

from lang_sam import LangSAM  # https://github.com/luca-medeiros/lang-segment-anything


def _to_numpy_mask(m):
    """Convert mask to uint8 [0,255] HxW array."""
    if isinstance(m, torch.Tensor):
        m = m.detach().cpu().numpy()
    else:
        m = np.array(m)
    m = np.squeeze(m)

    if m.dtype == bool:
        m = m.astype(np.uint8) * 255
    elif m.dtype != np.uint8:
        m = (m >= 0.5).astype(np.uint8) * 255
    return m


def main():
    # 1) load inputs from current directory
    image_pil = Image.open("sdxl_image_20251030_154354.png").convert("RGB")
    with open("mirror_prompt.txt", "r", encoding="utf-8") as f:
        prompts = [ln.strip() for ln in f if ln.strip()]
    if len(prompts) != 1:
        raise ValueError("mirror_prompt.txt must contain exactly ONE non-empty line.")
    text_prompt = prompts[0]

    # 2) init model
    model = LangSAM()

    # 3) predict
    results = model.predict([image_pil], [text_prompt])

    if not results or "masks" not in results[0] or len(results[0]["masks"]) == 0:
        raise RuntimeError("No mask returned by LangSAM for the given prompt/image.")

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # 4) save all masks
    for idx, m in enumerate(results[0]["masks"]):
        mask_np = _to_numpy_mask(m)
        out_path = f"mask_{idx}_{timestamp}.png"
        Image.fromarray(mask_np, mode="L").save(out_path)
        print(f"Saved {out_path}")


if __name__ == "__main__":
    main()


One or both local paths not provided. Loading from Hugging Face Hub: IDEA-Research/grounding-dino-base
Predicting 1 masks
Predicted 1 masks
Saved mask_0_20251030_160853.png
Saved mask_1_20251030_160853.png


/tmp/ipython-input-2001126203.py:49: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  Image.fromarray(mask_np, mode="L").save(out_path)
